## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-06-16-56-32 +0000,wapo,Five soldiers shot at Army’s Fort Stewart base...,https://www.washingtonpost.com/nation/2025/08/...
1,2025-08-06-16-56-15 +0000,nypost,Italy approves $15.5 billion project to build ...,https://nypost.com/2025/08/06/world-news/italy...
2,2025-08-06-16-53-57 +0000,nyt,"With Sanctions Looming, Trump’s Envoy to Russi...",https://www.nytimes.com/2025/08/06/world/europ...
3,2025-08-06-16-53-12 +0000,nyt,5 Soldiers Shot at Fort Stewart in Georgia,https://www.nytimes.com/2025/08/06/us/fort-ste...
4,2025-08-06-16-52-05 +0000,startribune,5 soldiers shot at Army’s Fort Stewart in Geor...,https://www.startribune.com/army-base-in-georg...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2369/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
21,trump,60
557,will,14
202,new,13
71,tariffs,13
72,india,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
47,2025-08-06-15-28-28 +0000,bbc,Trump-Modi ties hit rock bottom with new tarif...,https://www.bbc.com/news/articles/c15lqe7v302o...,120
211,2025-08-06-01-44-41 +0000,nypost,Trump envoy Witkoff urged to take tough approa...,https://nypost.com/2025/08/05/us-news/trump-en...,119
141,2025-08-06-10-11-50 +0000,nypost,Trump envoy Steve Witkoff meets Putin ahead of...,https://nypost.com/2025/08/06/world-news/trump...,114
172,2025-08-06-09-00-00 +0000,wsj,Democrats are adding potentially risky Epstein...,https://www.wsj.com/politics/policy/epstein-fi...,110
5,2025-08-06-16-48-00 +0000,wsj,Trump Envoy Witkoff Meets With Putin as Russia...,https://www.wsj.com/world/russia/trump-envoy-w...,109


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
47,120,2025-08-06-15-28-28 +0000,bbc,Trump-Modi ties hit rock bottom with new tarif...,https://www.bbc.com/news/articles/c15lqe7v302o...
152,60,2025-08-06-10-00-00 +0000,latimes,Chabria: Newsom vows Texas will be 'neutered' ...,https://www.latimes.com/california/story/2025-...
211,59,2025-08-06-01-44-41 +0000,nypost,Trump envoy Witkoff urged to take tough approa...,https://nypost.com/2025/08/05/us-news/trump-en...
172,46,2025-08-06-09-00-00 +0000,wsj,Democrats are adding potentially risky Epstein...,https://www.wsj.com/politics/policy/epstein-fi...
107,44,2025-08-06-12-45-15 +0000,nyt,"Alec Luhn, Missing American Journalist, Found ...",https://www.nytimes.com/2025/08/06/world/europ...
27,43,2025-08-06-15-57-37 +0000,nypost,Single mom knocked out by Cincinnati mob opens...,https://nypost.com/2025/08/06/us-news/single-m...
23,40,2025-08-06-16-14-13 +0000,nypost,At least five soldiers shot at Georgia’s Fort ...,https://nypost.com/2025/08/06/us-news/georgias...
134,39,2025-08-06-11-00-00 +0000,nypost,Zohran Mamdani’s Democratic Socialists of Amer...,https://nypost.com/2025/08/06/world-news/zohra...
244,38,2025-08-05-23-19-49 +0000,latimes,L.A. County residents illegally exported 'sens...,https://www.latimes.com/california/story/2025-...
322,30,2025-08-05-18-55-00 +0000,wsj,The top job at the Bureau of Labor Statistics ...,https://www.wsj.com/politics/help-wanted-a-195...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
